Template for Cleaning the Dataset

In [188]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from collections import Counter
from imblearn.over_sampling import SMOTE
import os
from datetime import datetime
from sklearn.decomposition import PCA

In [189]:
path = "Data/train"
list_files = list()
df = pd.DataFrame()
for file in os.listdir(path):
    if file.endswith('.csv'):
        list_files.append(file)
        
list_files

['19_7.csv',
 '19_1.csv',
 '19_2.csv',
 '19_3.csv',
 '35_1.csv',
 '35_2.csv',
 '34_3.csv',
 '34_1.csv',
 '36_3.csv']

In [190]:
list_files[4]

'35_1.csv'

In [191]:
df = pd.read_csv(path + '/' + list_files[4])
df.head()

,ID,Tune Date,System,Test ID,69 m/z,70 m/z,219 m/z,220 m/z,502 m/z,503 m/z,...,Emission,EM Volts,Ent Lens,Ion Focus,Repeller,Ent Lens offset,Filament,Width219,Temp,Condition
0,1,10/08/2020 2:34 PM (UTC+10:00),35,1,324224,3632,251008,11280,17232,1507,...,34.6,1621,20.18,90.3,27.73,14.273,1,-0.019,250,0
1,2,11/08/2020 11:40 AM (UTC+10:00),35,1,401600,4205,309376,13308,20704,2258,...,34.6,1650,20.18,90.3,28.52,14.273,1,-0.022,250,0
2,3,12/08/2020 9:23 AM (UTC+10:00),35,1,354432,3940,268032,11653,17904,1646,...,34.6,1641,20.18,90.3,28.13,14.273,1,-0.021,250,0
3,4,13/08/2020 11:45 AM (UTC+10:00),35,1,437312,5037,335168,14327,21016,2364,...,34.6,1680,22.67,90.3,28.13,14.110,1,-0.020,250,0
4,5,14/08/2020 3:00 PM (UTC+10:00),35,1,451200,4897,347840,14616,21864,2533,...,34.6,1687,22.67,90.3,28.52,14.110,1,-0.019,250,0


In [192]:
df.columns

Index(['ID', 'Tune Date', 'System', 'Test ID', '69 m/z', '70 m/z', '219 m/z',
       '220 m/z', '502 m/z', '503 m/z', 'Amu gain', 'Amu offset', 'Emission',
       'EM Volts', 'Ent Lens', 'Ion Focus', 'Repeller', 'Ent Lens offset',
       'Filament', 'Width219', 'Temp', 'Condition'],
      dtype='object')

* Clean inconsistant Tune Date

In [193]:
df.drop(['Filament'], axis=1, inplace=True) # Drop Filament column
df.drop(['Emission'], axis=1, inplace=True) # Drop Emission column
df.drop(['Temp'], axis=1, inplace=True) # Drop Temp column
df.drop(['Test ID'], axis=1, inplace=True) # Drop Test ID column

df['502_to_69_ratio'] = df['502 m/z'] / df['69 m/z'] # Create new column with ratio of 502 m/z to 69 m/z

############################################
#clean the datetime format
def clean_date(date):
    if 'UTC' in date:
        date = date.split('(')[0]
        date = date.split("  ")[0]
    return date

def convert_to_datetime(date):
    date_string = date
    date_format = '%d/%m/%Y %I:%M %p'

    datetime_obj = datetime.strptime(date_string, date_format)
    formatted_datetime = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_datetime

df['Tune Date'] = df['Tune Date'].apply(lambda x: clean_date(x))
df['Tune Date'] = df['Tune Date'].apply(lambda x: convert_to_datetime(x))
df['Tune Date'] = pd.to_datetime(df['Tune Date'])
df['total_hour_diff'] = df['Tune Date'].diff().apply(lambda x: x.total_seconds()/3600) # new column with total hour difference

df.drop(['Tune Date'], axis=1, inplace=True) # Drop Tune Date column
############################################


# Perform PCA on the highly correlated columns
pca = PCA(n_components=1)
components = pca.fit_transform(df[['69 m/z', '70 m/z']])
print(pca.explained_variance_ratio_)
df['PCA_Component 69 m/z'] = components

pca = PCA(n_components=1)
components = pca.fit_transform(df[['219 m/z', '220 m/z']])
print(pca.explained_variance_ratio_)
df['PCA_Component 219 m/z'] = components

pca = PCA(n_components=1)
components = pca.fit_transform(df[['502 m/z', '503 m/z']])
print(pca.explained_variance_ratio_)
df['PCA_Component 502 m/z'] = components


# add columns for pct change
def precent_change(col_name , periods):
    df[col_name + " pct change " + str(periods) + 'days'] = df[col_name].pct_change(periods=periods) * 100
    return df[col_name + " pct change " + str(periods) + 'days']

for i in range (5):
    precent_change('EM Volts', i+1)
for i in range (5):
    precent_change('502_to_69_ratio', i+1)

df.head()

,ID,Tune Date,System,69 m/z,70 m/z,219 m/z,220 m/z,502 m/z,503 m/z,Amu gain,Amu offset,EM Volts,Ent Lens,Ion Focus,Repeller,Ent Lens offset,Width219,Condition,502_to_69_ratio,total_hour_diff
0,1,2020-08-10 14:34:00,35,324224,3632,251008,11280,17232,1507,2656,133.625,1621,20.18,90.3,27.73,14.273,-0.019,0,0.053148,NaN
1,2,2020-08-11 11:40:00,35,401600,4205,309376,13308,20704,2258,2655,133.563,1650,20.18,90.3,28.52,14.273,-0.022,0,0.051554,21.100000
2,3,2020-08-12 09:23:00,35,354432,3940,268032,11653,17904,1646,2657,133.438,1641,20.18,90.3,28.13,14.273,-0.021,0,0.050515,21.716667
3,4,2020-08-13 11:45:00,35,437312,5037,335168,14327,21016,2364,2655,134.000,1680,22.67,90.3,28.13,14.110,-0.020,0,0.048057,26.366667
4,5,2020-08-14 15:00:00,35,451200,4897,347840,14616,21864,2533,2657,133.750,1687,22.67,90.3,28.52,14.110,-0.019,0,0.048457,27.250000
